<a href="https://colab.research.google.com/github/star-nox/Privacy-Policy-Summarization/blob/main/summarization_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
# Checking GPU availability
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.5 MB/s eta 0:00:00


In [8]:
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 26.1 MB/s eta 0:00:00


In [9]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=7b46131bba88d0d0c5ff1502e4a486aae04a4abe77b31926d5e094569b9a2504
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [2]:
# run this code when running the code on Google Colab
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/MyDrive/Applied_ML_Project/')

Mounted at /content/drive


In [4]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.8 MB/s eta 0:00:00


In [5]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
# importing required libraries
import pandas as pd
import transformers
from datasets import load_metric
import re

from sklearn.model_selection import train_test_split


In [11]:
# read pickle file

data = pd.read_pickle("/content/drive/MyDrive/Applied_ML_Project/data_preprocessing/summary_data1.pkl")
data.head()

,original_text,reference_summary
0,"On September 15, 2005, the Equal Employment Op...",Equal Employment Opportunity Commission brough...
1,NOTE: This is one of three identically named c...,The case was brought by a non-profit organizat...
2,"On May 11, 2006, African-American employees of...",This case was brought by African American empl...
3,Pursuant to the Civil Rights of Institutionali...,Pursuant to the Civil Rights of Institutionali...
4,"On July 30, 2015, the Freedom of the Press Fou...",A non-profit organization dealing with rights ...


In [12]:
data.dropna(how='any', inplace = True)

In [10]:
len(data)

30866

In [13]:
# selecting only those rows with string length <= 1024

new_data = data[data['original_text'].apply(lambda x: len(x.split(' ')) <= 1024)]
new_data.head()

,original_text,reference_summary
0,"On September 15, 2005, the Equal Employment Op...",Equal Employment Opportunity Commission brough...
1,NOTE: This is one of three identically named c...,The case was brought by a non-profit organizat...
2,"On May 11, 2006, African-American employees of...",This case was brought by African American empl...
3,Pursuant to the Civil Rights of Institutionali...,Pursuant to the Civil Rights of Institutionali...
4,"On July 30, 2015, the Freedom of the Press Fou...",A non-profit organization dealing with rights ...


In [14]:
new_data.iloc[0,0]

'On September 15, 2005, the Equal Employment Opportunity Commission (EEOC) filed suit against House of Philadelphia, Inc., on behalf of an employee who was allegedly fired because she was pregnant. Seeking monetary and injunctive relief for the employee (including economic damage, compensation for emotional harm, and punitive damages), the EEOC brought suit under Title VII of the Civil Rights Act of 1964 for unlawful discrimination on the basis of sex. The EEOC also sought to recover its costs. Via private counsel, the employee filed a motion to intervene in the suit, which was automatically granted after the period for filing objections passed without incident. The employee brought claims under Title VII and state law and sought substantially the same relief as the EEOC, except that the complaint specifically sought reinstatement. Eventually the parties came to a settlement agreement, which the Court (Judge Kristi K. DuBose) entered as a consent decree on Jan 10, 2009. The terms of th

In [15]:
train_data, val_test_data = train_test_split(new_data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(val_test_data, test_size=0.5, random_state=123)

In [35]:
len(train_data)

11193

Finetuning t5 model

In [16]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00


In [17]:
from transformers import AutoTokenizer

#model_checkpoint = "t5-small"
model_checkpoint = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [17]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8087, 108, 136, 156, 5577, 147, 1], [182, 117, 372, 5577, 107, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [18]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this a sentence!", "This is another sentence."]))

{'input_ids': [[8087, 108, 136, 114, 5577, 147, 1], [182, 117, 372, 5577, 107, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [19]:
max_input_length = 1024
max_target_length = 256

def preprocess(df):
    tokenized_data = []
    for index, row in df.iterrows():
        inputs = re.sub("\s\s+", " ", row['original_text'])    
        model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
        # Setup the tokenizer for targets
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(row['reference_summary'], max_length=max_target_length, truncation=True)

        model_inputs['labels'] = labels['input_ids']
        tokenized_data.append(model_inputs)
    return tokenized_data

In [20]:
tokenized_train_data = preprocess(train_data)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [22]:
print(tokenized_train_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
tokenized_val_data = preprocess(val_data)

In [22]:
tokenized_test_data = preprocess(test_data)

In [23]:
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [24]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-policy",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [26]:
import nltk
import numpy as np

metric = load_metric("rouge")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

<ipython-input-26-068a0a7fb513>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [27]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/star-nox/pegasus-cnn_dailymail-finetuned-policy into local empty directory.


In [29]:
trainer.train()

OutOfMemoryError: ignored

In [ ]:
trainer.push_to_hub()
